# Supervisor

![Supervisor Multi Agent](images/supervisor_multi_agent.png)

## Single Agent

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display

@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}

system_message = """
You are a financial advisor assistant. Use the provided tools to ground your answers
in up-to-date market data. Be concise, factual, and risk-aware.

Be decisive: when you have sufficient information to act, proceed with tool calls without
asking for confirmation. Only if information is missing or uncertain, ask a concise 
clarifying question.

When preparing or describing actions, include appropriate parameters (e.g., symbol, shares,
limit price, budgets) based on available data. Do not fabricate numbers or facts.
"""

@tool
def place_order(
    symbol: str,
    action: str,
    shares: int,
    limit_price: float,
    order_type: str = "limit",
) -> dict:
    """
    Execute a stock order.

    Parameters:
    - symbol: Ticker
    - action: "buy" or "sell"
    - shares: Number of shares to trade (pre-computed by the agent)
    - limit_price: Limit price per share
    - order_type: Order type, default "limit"

    Returns:
    - status: Execution result (simulated)
    - symbol
    - shares
    - limit_price
    - total_spent
    - type: Order type used
    - action
    """
    total_spent = round(int(shares) * limit_price, 2)
    return {
        "status": "filled",
        "symbol": symbol,
        "shares": int(shares),
        "limit_price": limit_price,
        "total_spent": total_spent,
        "type": order_type,
        "action": action,
    }

agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[lookup_stock_symbol, fetch_stock_data_raw, place_order],
    prompt=system_message
)

display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
response = agent.invoke({"messages": [HumanMessage(content="Buy $1000 of Tesla stock at the current price.")]})
for message in response['messages']:
    message.pretty_print()

In [ ]:
response = agent.invoke({"messages": [HumanMessage(content="""
I’m thinking of investing $1,000 in the electric vehicle industry. 
Could you find the most promising companies right now and explain why they might be a good choice?
""")]})
for message in response['messages']:
    message.pretty_print()

### Adding Research Part

In [ ]:
from typing import List, Dict
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage

system_message = """
You are a financial advisor assistant. Use the available tools to ground answers in up-to-date, factual information. Be concise, risk-aware, and action-oriented.

Planning rubric:
- If the request is thematic or ambiguous (e.g., “invest in AI”), first do discovery: find candidates, validate relevance, and pick ONE company. Keep research tight (2–3 calls total). Avoid fetching exact trading numbers during research.
- If the request names a specific company, skip discovery and move to execution.
- For execution: determine the correct ticker, fetch current market data, size the order from the user’s budget, and place a buy order when appropriate.

Behavior:
- Be decisive: when you have enough information, proceed with tool calls without asking for confirmation. Ask a brief clarifying question only if critical details are missing (e.g., budget, company/theme not clear).
- Do not fabricate numbers or facts. Prefer small, reliable steps over speculation.
- Minimize redundant calls (e.g., don’t fetch the same data repeatedly without reason).
- Translate data into plain language. Keep outputs short unless the user asks for details.

Outputs:
- For research requests: 1–2 sentences explaining the chosen company and why it fits the user’s request; then continue with the execution flow automatically if budget is provided.
- For execution requests: briefly state what you’re doing (ticker, rough rationale) and the final action taken (order placed, with key parameters).
"""

FORBIDDEN_KEYWORDS = {
    "403 forbidden", "access denied", "captcha",
    "has been denied", "not authorized", "verify you are a human"
}

@tool
def web_search(query: str, max_results: int = 5) -> Dict[str, List[Dict[str, str]]]:
    """
    General-purpose web search.

    Use when you need recent or broader information from the web to answer the user's request
    (e.g., discover relevant entities, find supporting context, or gather up-to-date references).

    Parameters:
    - query (str): The search query in plain language.
    - max_results (int): Number of results to return (default 5, max 10).

    Returns:
    - {"results": [{"title": str, "url": str, "snippet": str}, ...]}

    Example:
    - query: "emerging AI hardware companies"
    """
    max_results = max(1, min(max_results, 10))
    tavily = TavilySearch(max_results=max_results)
    raw = tavily.invoke({"query": query})

    results = [
        {k: v for k, v in page.items() if k != "raw_content"}  # drop heavy field
        for page in raw["results"]
        if not any(
            k in ((page.get("content") or "").lower())
            for k in FORBIDDEN_KEYWORDS
        )
    ]

    return {"results": results}


from langchain_community.document_loaders import WikipediaLoader

@tool
def wiki_search(topic: str, max_results: int = 5) -> dict:
    """
    Fetch a concise encyclopedic summary for a single entity or topic.

    When to use:
      - You need neutral background about a company, product, person, or concept.

    How to format `topic` (VERY IMPORTANT):
      - Pass a short, Wikipedia-friendly title or entity name.
      - Avoid questions or long queries. Prefer canonical forms.
      - If you have noisy text, reduce it to the key noun phrase.

    Good examples:
      - "NVIDIA", "OpenAI", "Large language model", "Electric vehicle"
    Avoid:
      - "What is NVIDIA and why is it important?", "tell me about AI chips 2025"

    Parameters:
      - topic (str): Canonical page title or concise entity/topic.
    """
    max_results = max(1, min(max_results, 10))
    wiki = WikipediaLoader(query=topic, load_max_docs=max_results)
    raw = wiki.load()

    results = [
      {
        "title": doc.metadata["title"],
        "summary": doc.metadata["summary"],
        "source": doc.metadata["source"]
      }
      for doc in raw
    ]

    return {"results": results}


agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, wiki_search, lookup_stock_symbol, fetch_stock_data_raw, place_order],
    prompt=system_message
)

display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
response = agent.invoke({"messages": [HumanMessage(content="""
I’m thinking of investing $1,000 in the electric vehicle industry. 
Could you find the most promising companies right now and explain why they might be a good choice?
""")]})
for message in response['messages']:
    message.pretty_print()

### Introducing NOW

In [ ]:
from datetime import datetime, timezone

now = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

response = agent.invoke({"messages": [
    SystemMessage(content=f"Today's date is {now}. Treat this as NOW"),
    HumanMessage(content="""
    I want you to invest $1,000 into the most promising company in the renewable energy sector.  
    Please research the options, pick the best candidate, and then go ahead and place a buy order for me.
    """)
]})
for message in response['messages']:
    message.pretty_print()

## Supervisor

### research_agent

In [ ]:
from typing import List, Dict
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage


research_system_message = """
You are a Research Agent specializing in identifying one promising company for potential investment based on the user’s request.

Responsibilities:
- Interpret the user’s theme or sector (e.g., AI, renewable energy, EVs) and propose ONE company that best fits.
- Use the available tools to discover, verify, and cross-check information.
- Prefer recent, credible information and avoid speculation.

Behavior:
- Do NOT place or simulate trades. Your job ends at recommending a company.
- Keep research tight (aim for 2–3 tool calls); refine queries if results are noisy.
- Ask a brief clarifying question only if the request is too vague to proceed.
- Do not fabricate numbers or facts. Be concise, neutral, and risk-aware.
- Consider the current date when discussing recency or momentum.

Outputs:
- 1–2 sentences explaining why the company fits the request (clear, plain language).
- Final line: CHOSEN_COMPANY: <Company Name>
"""

FORBIDDEN_KEYWORDS = {
    "403 forbidden", "access denied", "captcha",
    "has been denied", "not authorized", "verify you are a human"
}

@tool
def web_search(query: str, max_results: int = 5) -> Dict[str, List[Dict[str, str]]]:
    """
    General-purpose web search.

    Use when you need recent or broader information from the web to answer the user's request
    (e.g., discover relevant entities, find supporting context, or gather up-to-date references).

    Parameters:
    - query (str): The search query in plain language.
    - max_results (int): Number of results to return (default 5, max 10).

    Returns:
    - {"results": [{"title": str, "url": str, "snippet": str}, ...]}

    Example:
    - query: "emerging AI hardware companies"
    """
    max_results = max(1, min(max_results, 10))
    tavily = TavilySearch(max_results=max_results)
    raw = tavily.invoke({"query": query})

    results = [
        {k: v for k, v in page.items() if k != "raw_content"}  # drop heavy field
        for page in raw["results"]
        if not any(
            k in ((page.get("content") or "").lower())
            for k in FORBIDDEN_KEYWORDS
        )
    ]

    return {"results": results}


from langchain_community.document_loaders import WikipediaLoader

@tool
def wiki_search(topic: str, max_results: int = 5) -> dict:
    """
    Fetch a concise encyclopedic summary for a single entity or topic.

    When to use:
      - You need neutral background about a company, product, person, or concept.

    How to format `topic` (VERY IMPORTANT):
      - Pass a short, Wikipedia-friendly title or entity name.
      - Avoid questions or long queries. Prefer canonical forms.
      - If you have noisy text, reduce it to the key noun phrase.

    Good examples:
      - "NVIDIA", "OpenAI", "Large language model", "Electric vehicle"
    Avoid:
      - "What is NVIDIA and why is it important?", "tell me about AI chips 2025"

    Parameters:
      - topic (str): Canonical page title or concise entity/topic.
    """
    max_results = max(1, min(max_results, 10))
    wiki = WikipediaLoader(query=topic, load_max_docs=max_results)
    raw = wiki.load()

    results = [
      {
        "title": doc.metadata["title"],
        "summary": doc.metadata["summary"],
        "source": doc.metadata["source"]
      }
      for doc in raw
    ]

    return {"results": results}


research_agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, wiki_search],
    prompt=research_system_message,

    name="research_agent"
)

display(Image(research_agent.get_graph().draw_mermaid_png()))

### trading_agent

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from IPython.display import Image, display

@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}

trading_system_message = """
You are a financial advisor assistant. Use the provided tools to ground your answers
in up-to-date market data. Be concise, factual, and risk-aware.

Be decisive: when you have sufficient information to act, proceed with tool calls without
asking for confirmation. Only if information is missing or uncertain, ask a concise 
clarifying question.

When preparing or describing actions, include appropriate parameters (e.g., symbol, shares,
limit price, budgets) based on available data. Do not fabricate numbers or facts.
"""

@tool
def place_order(
    symbol: str,
    action: str,
    shares: int,
    limit_price: float,
    order_type: str = "limit",
) -> dict:
    """
    Execute a stock order.

    Parameters:
    - symbol: Ticker
    - action: "buy" or "sell"
    - shares: Number of shares to trade (pre-computed by the agent)
    - limit_price: Limit price per share
    - order_type: Order type, default "limit"

    Returns:
    - status: Execution result (simulated)
    - symbol
    - shares
    - limit_price
    - total_spent
    - type: Order type used
    - action
    """
    total_spent = round(int(shares) * limit_price, 2)
    return {
        "status": "filled",
        "symbol": symbol,
        "shares": int(shares),
        "limit_price": limit_price,
        "total_spent": total_spent,
        "type": order_type,
        "action": action,
    }

trading_agent = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[lookup_stock_symbol, fetch_stock_data_raw, place_order],
    prompt=trading_system_message,

    name="trading_agent"
)

display(Image(trading_agent.get_graph().draw_mermaid_png()))

### supervisor

In [ ]:
from IPython.display import Image, display
from langgraph_supervisor import create_supervisor
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.tools import tool

@tool
def current_timestamp() -> dict:
    """
    Return the current local timestamp.

    Returns:
    - {"iso": str, "epoch": int, "tz": str}
      where:
      - iso: ISO 8601 string with timezone offset
      - epoch: Unix epoch seconds
      - tz: timezone name/offset
    """
    now = datetime.now().astimezone()
    return {
        "iso": now.isoformat(),
        "epoch": int(now.timestamp()),
        "tz": str(now.tzinfo),
    }

supervisor_system_message = """
You are a Supervisor coordinating two specialists:

- research_agent: finds ONE suitable company matching the user’s request and explains why.
- trading_agent: given a company and a budget/action, determines the ticker, checks market data, sizes the order, and places a trade.

Your goal is to satisfy the user’s intent with minimal steps.

Clocking / Context:
- If the conversation does not already contain a “NOW” context, first obtain it by calling the tool `current_timestamp`.
- After obtaining it, post a single one-line note into the thread so it’s available to all subsequent steps, e.g.:
  "System context — NOW: {iso} ({tz}); epoch={epoch}"
- Use this “NOW” as the reference for recency. Do not call `current_timestamp` again unless the prior “NOW” is missing or clearly stale.


Routing:
- If the request is thematic or ambiguous, ask research_agent.
- If the request already names a company and includes an action/budget, use trading_agent.
- If a single key detail is missing (e.g., budget), ask once, then proceed.

Handoff:
- From research_agent expect: one company name + 1–2 sentence rationale.
- Pass that company (and any provided budget/action) to trading_agent.

Guardrails:
- Don’t invent data. Don’t place trades without explicit user budget/action.
- Prefer the current date context when judging recency.

Output:
- Briefly state what you delegated and the result. If blocked, ask only for what’s needed to proceed.
"""

supervisor = create_supervisor(
    agents=[research_agent, trading_agent],
    tools=[current_timestamp],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=supervisor_system_message,
    
    output_mode="full_history"
).compile()

display(Image(supervisor.get_graph().draw_mermaid_png()))

In [ ]:
display(Image(supervisor.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
response = supervisor.invoke({"messages": [
    HumanMessage(content="""I want you to invest $1,000 into the most promising company in the AI sector.  
    Please research the options, pick the best candidate, and then go ahead and place a buy order for me.
    """)
]})
for message in response['messages']:
    message.pretty_print()

## Reference Links

**1. LangGraph Supervisor: API Reference**

https://langchain-ai.github.io/langgraph/reference/supervisor/

→ Technical reference for supervisor utilities (create_supervisor, handoff/forward tools), parameters, and options.

**2. Multi-Agent Supervisor Tutorial**

https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor/

→ Step-by-step guide to building a supervisor-led multi-agent system: create worker agents, add a supervisor (package or from scratch), and delegate tasks.

**3. Multi-Agent Concepts in LangGraph**

https://langchain-ai.github.io/langgraph/concepts/multi_agent/

→ Conceptual overview of multi-agent patterns (handoffs, networks, supervisor/hierarchical), communication, and state management.

**4. langgraph-supervisor-py: GitHub Repository**

https://github.com/langchain-ai/langgraph-supervisor-py

→ Official package repository with features, installation, quickstart, and customization guides (handoffs, message forwarding).
